In [1]:
from google.colab import drive
drive.mount('drive')
import pandas as pd
import numpy as np
diabetes_df = pd.read_csv('/content/drive/MyDrive/CL/Day 4/diabetes.csv', sep=',' )

diabetes_df

Mounted at drive


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [2]:
!pip install ydata_profiling

import ydata_profiling as ydp

diabetes_report = ydp.ProfileReport(diabetes_df)
diabetes_report


Output hidden; open in https://colab.research.google.com to view.

In [3]:
diabetes_df.isnull().sum()


,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [5]:
from sklearn.model_selection import train_test_split

# Podział danych na zmienne niezależne (X) i zależną (y)
X = diabetes_df.drop(columns=['Outcome'])
y = diabetes_df['Outcome']

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [6]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [9]:
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic')
params = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.3],
    'learning_rate': [0.01, 0.1, 0.3]
}
grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=params, cv=3, n_jobs=-1, verbose=2, scoring='precision')


In [10]:
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

model = grid_search.best_estimator_

y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred))

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best Parameters: {'gamma': 0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.65      0.96      0.77        94
           1       0.73      0.18      0.29        60

    accuracy                           0.66       154
   macro avg       0.69      0.57      0.53       154
weighted avg       0.68      0.66      0.59       154

